### ETL and SQL Window Function Practice Problems

#### Problem 1: Extract and Clean Data
**Task:**
1. Extract data from the `customers` table.
2. Clean the data by:
   - Removing customers who have not provided a `PostalCode`.
   - Standardizing the `Country` column to use uppercase.


In [39]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

In [41]:
engine = create_engine("sqlite:///Chinook_Sqlite.sqlite")
connection = engine.connect()

In [40]:
def sql_query(sql_query_text,show_result=False,connection=connection):
    df = pd.read_sql_query(sql_query_text,con=connection)
    if show_result:
        display(df.head())
    return df

def sql_command(sql_query_text,connection=connection):
    try:
        connection.execute(text(sql_query_text))
        connection.commit()
        print("Command executed successfully.")

    except SQLAlchemyError as e:
        connection.rollback()
        print(f"An error occurred: {str(e)}")


In [15]:
df = sql_query("SELECT * FROM customer where PostalCode is not null and PostalCode!=''")
df['Country'] = df['Country'].str.upper()

df.head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,BRAZIL,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,GERMANY,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,CANADA,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,NORWAY,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,CZECH REPUBLIC,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [ ]:
sql_query("select name from sqlite_master where type='table'")

,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


In [ ]:
sql_query("pragma table_info('Customer')")

,cid,name,type,notnull,dflt_value,pk
0,0,CustomerId,INTEGER,1,None,1
1,1,FirstName,NVARCHAR(40),1,None,0
2,2,LastName,NVARCHAR(20),1,None,0
3,3,Company,NVARCHAR(80),0,None,0
4,4,Address,NVARCHAR(70),0,None,0
5,5,City,NVARCHAR(40),0,None,0
6,6,State,NVARCHAR(40),0,None,0
7,7,Country,NVARCHAR(40),0,None,0
8,8,PostalCode,NVARCHAR(10),0,None,0
9,9,Phone,NVARCHAR(24),0,None,0


In [ ]:
sql_query("pragma table_info('Invoice')")

,cid,name,type,notnull,dflt_value,pk
0,0,InvoiceId,INTEGER,1,None,1
1,1,CustomerId,INTEGER,1,None,0
2,2,InvoiceDate,DATETIME,1,None,0
3,3,BillingAddress,NVARCHAR(70),0,None,0
4,4,BillingCity,NVARCHAR(40),0,None,0
5,5,BillingState,NVARCHAR(40),0,None,0
6,6,BillingCountry,NVARCHAR(40),0,None,0
7,7,BillingPostalCode,NVARCHAR(10),0,None,0
8,8,Total,"NUMERIC(10,2)",1,None,0


#### Problem 2: Data Transformation and Aggregation
**Task:**
1. Extract data from the `invoices` and `customers` tables.
2. Merge the data to calculate the total amount spent by each customer.
3. Rank the customers based on their total spending.

In [36]:
sql = """
select c.CustomerId, SUM(i.Total) as customer_total, 
        RANK() OVER(
                ORDER BY SUM(i.Total) DESC
            ) as rank_col
    from Invoice i
        join Customer c on c.CustomerId = i.CustomerId
        group by c.CustomerId
        order by rank_col asc
"""
df = sql_query(sql)
df

,CustomerId,customer_total,rank_col
0,6,49.62,1
1,26,47.62,2
2,57,46.62,3
3,45,45.62,4
4,46,45.62,4
5,24,43.62,6
6,28,43.62,6
7,37,43.62,6
8,7,42.62,9
9,25,42.62,9



#### Problem 3: Load Transformed Data Back into the Database
**Task:**
1. Create a new table in the database called `customer_spending` and load the `customer_total_spending` DataFrame into it.


In [42]:
sql = """
create table if not exists customer_spending (
    Id INTEGER PRIMARY KEY AUTOINCREMENT,
    CustomerId	INTEGER NOT NULL,
    CustomerTotal NUMERIC(10,2) default 0,
    RankCol INTEGER default 0
)
"""
sql_command(sql)

Command executed successfully.


In [46]:
df_to_insert = df[['CustomerId', 'customer_total', 'rank_col']].rename(
    columns={'customer_total': 'CustomerTotal', 'rank_col': 'RankCol'}
)
df_to_insert.to_sql('customer_spending',con=connection, if_exists='replace', index=False)

59

#### Problem 4: SQL Window Functions
**Task:**
1. Using SQL, calculate the cumulative total (`running total`) of all invoices in the `invoices` table, ordered by `InvoiceDate`.
2. For each invoice, find the rank of the invoice amount within each customer's invoices using `RANK()` window function.


### Problem 4: Comprehensive Window Function Practice

#### 4a) **Ranking Functions**
**Use Case:** Ranking customers and albums based on specific criteria.

**Tasks:**
1. Rank customers by the total amount they have spent within each country (`Customer.Country`). Use a ranking function to generate ranks for customers in descending order of their total spending.
2. Rank albums based on the total number of tracks they contain. Use a dense ranking function to avoid gaps in the ranking sequence when albums have the same number of tracks.

In [55]:
sql = """ 
select c.CustomerId, c.Country, SUM(i.Total) as customer_country_total,
    RANK() OVER ( PARTITION BY c.Country ORDER BY SUM(i.Total) DESC ) as rank_col
    from Customer c
        join Invoice i on i.CustomerId = c.CustomerId
    group by c.CustomerId, c.Country
"""
sql_query(sql)

,CustomerId,Country,customer_country_total,rank_col
0,56,Argentina,37.62,1
1,55,Australia,37.62,1
2,7,Austria,42.62,1
3,8,Belgium,37.62,1
4,1,Brazil,39.62,1
5,10,Brazil,37.62,2
6,11,Brazil,37.62,2
7,12,Brazil,37.62,2
8,13,Brazil,37.62,2
9,3,Canada,39.62,1


In [56]:
sql = """ 
    select a.AlbumId,  COUNT(t.TrackId) as track_count,
        DENSE_RANK() OVER (ORDER BY COUNT(t.TrackId) DESC) as album_track_rank
    from Album a
        join Track t on t.AlbumId = a.AlbumId
        group by a.AlbumId
        order by album_track_rank
"""
sql_query(sql)

,AlbumId,track_count,album_track_rank
0,141,57,1
1,23,34,2
2,73,30,3
3,229,26,4
4,230,25,5
...,...,...,...
342,343,1,29
343,344,1,29
344,345,1,29
345,346,1,29


#### 4b) **Aggregate Functions**
**Use Case:** Calculating cumulative totals and moving averages.

**Tasks:**
1. Calculate the cumulative total (`running total`) of invoice amounts in the `invoices` table, ordered by `InvoiceDate`. This will help understand the total revenue generated over time.
2. Compute a moving average of track lengths within each album, considering only the current and previous two tracks ordered by `TrackId`. This will give an idea of how track lengths vary across an album.

In [57]:
sql = """ 
select i.InvoiceId, SUM(i.Total) OVER( ORDER BY i.InvoiceDate) as running_total
    from Invoice i
    order by i.InvoiceDate
"""
sql_query(sql)

,InvoiceId,running_total
0,1,1.98
1,2,5.94
2,3,11.88
3,4,20.79
4,5,34.65
...,...,...
407,408,2297.90
408,409,2303.84
409,410,2312.75
410,411,2326.61


In [59]:
sql_query("pragma table_info('Track')")

,cid,name,type,notnull,dflt_value,pk
0,0,TrackId,INTEGER,1,None,1
1,1,Name,NVARCHAR(200),1,None,0
2,2,AlbumId,INTEGER,0,None,0
3,3,MediaTypeId,INTEGER,1,None,0
4,4,GenreId,INTEGER,0,None,0
5,5,Composer,NVARCHAR(220),0,None,0
6,6,Milliseconds,INTEGER,1,None,0
7,7,Bytes,INTEGER,0,None,0
8,8,UnitPrice,"NUMERIC(10,2)",1,None,0


In [66]:
sql = """ 
select t.TrackId, 
    t.AlbumId, 
    t.Milliseconds,
    AVG(t.Milliseconds) OVER (
        PARTITION BY t.AlbumId
        ORDER BY t.TrackId
        ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
    ) as moving_avg

from Track t
order by t.AlbumId, t.TrackId
"""
sql_query(sql)

,TrackId,AlbumId,Milliseconds,moving_avg
0,1,1,343719,343719.000000
1,6,1,205662,274690.500000
2,7,1,233926,261102.333333
3,8,1,210834,216807.333333
4,9,1,203102,215954.000000
...,...,...,...,...
3498,3499,343,286741,286741.000000
3499,3500,344,139200,139200.000000
3500,3501,345,66639,66639.000000
3501,3502,346,221331,221331.000000



#### 4c) **Value Functions**
**Use Case:** Accessing specific rows within a window.

**Tasks:**
1. For each album, identify the highest and lowest track prices using the `FIRST_VALUE()` and `LAST_VALUE()` functions, ordered by price. This will help you find the price range of tracks within albums.
2. Use the `LEAD()` and `LAG()` functions to find the previous and next invoices for each customer, based on invoice date. This will show the interval between their purchases.

In [67]:
sql_query("pragma table_info('Track')")

,cid,name,type,notnull,dflt_value,pk
0,0,TrackId,INTEGER,1,None,1
1,1,Name,NVARCHAR(200),1,None,0
2,2,AlbumId,INTEGER,0,None,0
3,3,MediaTypeId,INTEGER,1,None,0
4,4,GenreId,INTEGER,0,None,0
5,5,Composer,NVARCHAR(220),0,None,0
6,6,Milliseconds,INTEGER,1,None,0
7,7,Bytes,INTEGER,0,None,0
8,8,UnitPrice,"NUMERIC(10,2)",1,None,0


In [73]:
sql = """ 
SELECT 
    a.AlbumId, t.TrackId,
    MIN(t.UnitPrice) OVER (PARTITION BY t.AlbumId) AS low_price,
    MAX(t.UnitPrice) OVER (PARTITION BY t.AlbumId) AS high_price
FROM 
    Album a
JOIN 
    Track t ON t.AlbumId = a.AlbumId;
"""
df = sql_query(sql)
df.head(30)

,AlbumId,TrackId,low_price,high_price
0,1,1,0.99,0.99
1,1,6,0.99,0.99
2,1,7,0.99,0.99
3,1,8,0.99,0.99
4,1,9,0.99,0.99
5,1,10,0.99,0.99
6,1,11,0.99,0.99
7,1,12,0.99,0.99
8,1,13,0.99,0.99
9,1,14,0.99,0.99


In [80]:
sql = """ 
select c.CustomerId, i.InvoiceId, i.InvoiceDate,
LAG(i.InvoiceDate) OVER (PARTITION BY c.CustomerId ORDER BY i.InvoiceDate) as prior_invoice_date,
LEAD(i.InvoiceDate) OVER (PARTITION BY c.CustomerId ORDER BY i.InvoiceDate) as next_invoice_date
from Invoice i
    join Customer c
    on c.CustomerId = i.CustomerId
"""
sql_query(sql)

,CustomerId,InvoiceId,InvoiceDate,prior_invoice_date,next_invoice_date
0,1,98,2010-03-11 00:00:00,None,2010-06-13 00:00:00
1,1,121,2010-06-13 00:00:00,2010-03-11 00:00:00,2010-09-15 00:00:00
2,1,143,2010-09-15 00:00:00,2010-06-13 00:00:00,2011-05-06 00:00:00
3,1,195,2011-05-06 00:00:00,2010-09-15 00:00:00,2012-10-27 00:00:00
4,1,316,2012-10-27 00:00:00,2011-05-06 00:00:00,2012-12-07 00:00:00
...,...,...,...,...,...
407,59,45,2009-07-08 00:00:00,2009-04-05 00:00:00,2010-02-26 00:00:00
408,59,97,2010-02-26 00:00:00,2009-07-08 00:00:00,2011-08-20 00:00:00
409,59,218,2011-08-20 00:00:00,2010-02-26 00:00:00,2011-09-30 00:00:00
410,59,229,2011-09-30 00:00:00,2011-08-20 00:00:00,2012-05-30 00:00:00


#### 4d) **Analytic Functions**
**Use Case:** Calculating ratios, percentiles, and other statistical insights.

**Tasks:**
1. Calculate the relative rank (percentile) of each invoice amount in the dataset using the PERCENT_RANK() function. This will help identify the position of each invoice in relation to others based on the total amount, providing insights into the distribution of revenue.
2. Divide each customer's invoices into quartiles based on their total amount using the NTILE() function. This will categorize invoices into four groups for each customer, ordered by the invoice total, providing insights into how each invoice ranks within the customer’s transaction history.

These problems will require the use of all four categories of window functions, providing a comprehensive practice set using the Chinook database. You can now implement each of these tasks to get a thorough understanding of how window functions operate and can be used to derive valuable insights from a dataset.

In [86]:
sql = """ 
select i.InvoiceId, i.Total,
    PERCENT_RANK() OVER (ORDER BY i.Total) as per_total
    from Invoice i
    order by i.Total
"""
sql_query(sql)

,InvoiceId,Total,per_total
0,6,0.99,0.000000
1,13,0.99,0.000000
2,20,0.99,0.000000
3,27,0.99,0.000000
4,34,0.99,0.000000
...,...,...,...
407,201,18.86,0.987835
408,96,21.86,0.992701
409,194,21.86,0.992701
410,299,23.86,0.997567


In [88]:
sql = """ 
select i.CustomerId, i.InvoiceId, i.Total,
    NTILE(4) OVER (PARTITION BY i.CustomerId ORDER BY i.Total) AS quartile
    from Invoice i
    order by i.CustomerId, i.Total
"""
df = sql_query(sql)
df.head(30)

,CustomerId,InvoiceId,Total,quartile
0,1,195,0.99,1
1,1,316,1.98,1
2,1,121,3.96,2
3,1,98,3.98,2
4,1,143,5.94,3
5,1,382,8.91,3
6,1,327,13.86,4
7,2,293,0.99,1
8,2,1,1.98,1
9,2,196,1.98,2


In [ ]:
connection.close()